### 0. Imports

In [228]:
import pandas as pd
import yfinance as yf
import numpy as np
import xlrd
from itertools import combinations

### 1. Data download

In [148]:
# read ticker data as Series and convert to list
tickers = pd.read_csv('tickers.csv', squeeze=True)
tickers = tickers.tolist()

In [149]:
price_list = []

for ticker in tickers:
    stock = yf.Ticker(ticker).history(period="3mo").reset_index()
    stock = stock[['Date', 'Close']]
    stock.rename(columns={'Close': ticker}, inplace=True)
    price_list.append(stock)

- BFB: No data found, symbol may be delisted


### 2. Compute ratios

In [261]:
# concatenate all prices into one big dataframe
df = pd.concat(price_list, axis=1)
df = df.loc[:,~df.columns.duplicated()].copy()

In [271]:
# remove the Date column for the computation
no_date_df = df.drop(labels='Date', axis=1)

In [308]:
# compute all ratios
ratio_df = pd.concat([no_date_df[no_date_df.columns.difference([col])].div(no_date_df.where(df !=0, np.nan)[col], axis=0)\
                           .add_suffix("_" + col) for col in no_date_df.columns], axis=1)

In [309]:
# add back 'Date' column...
ratio_df = pd.concat([ratio_df, price_list[0]['Date']], axis=1)
# ...and move it to the front of the df
first_column = ratio_df.pop('Date')
ratio_df.insert(0, 'Date', first_column)

/Users/rene/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3377: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if (await self.run_code(code, result,  async_=asy)):


In [ ]:
df_new.rename_axis("Date", axis=1)
df_delta = df_new.pct_change(periods=-1)
df_delta = df_delta.iloc[:4]
df_newstocks = df_delta[df_delta.iloc[:] >= 0]
positiveRatiosPercent = df_newstocks.drop(df_newstocks.columns[df_newstocks.apply(lambda col: col.isnull().sum() > 0)], axis=1)
labelList = list(positiveRatiosPercent.columns.values)
df_positiveRatios = df_new[labelList]
df_positiveRatios.rename(columns = {"Unnamed: 0" : "Dates"}, inplace = True)
df_positiveRatios.to_excel("path to excel file on the local computer")